### アプローチリストに使用するサンプルデータの生成

    それぞれのデータを生成し、個人情報はマスク化の処理を行う（厳密な方法ではない）

    ・契約データ：主人公となるデータ

    ・スコアリングデータ：webサイト未登録者に対してスコアリング（webサイトに登録してくれるであろう確率）を付与したデータ

    ・直近のwebサイト登録者：直近webサイトに登録した契約者のリスト

In [1]:
import pandas as pd
import numpy as np
import hashlib

In [2]:
# SHA-256でハッシュ化する関数
def hash_mask(value):
    hashed_value = hashlib.sha256(str(value).encode()).hexdigest()
    return hashed_value[:10]  # ハッシュ化された文字列の先頭10桁をマスク化に使用

## データを生成

In [3]:
# シード値を指定
seed_value = 1
np.random.seed(seed_value)


# 生データの生成
num_contracts = 15000  # 主契約データの件数
num_customers = 8000  # 契約者の人数（基本契約番号のユニーク数）
num_scores = 5000     # スコアリングデータの件数
num_registered = 2500 # Webサイト登録者数


# 基本契約番号（12桁のランダムな整数）、証券番号（11桁のランダムな整数）を生成
basic_contract_numbers = np.random.randint(100000000000, 999999999999, num_customers)
basic_contract_numbers = np.repeat(basic_contract_numbers, np.random.randint(1, 4, num_customers))[:num_contracts]  # 1人の顧客が1〜3つの契約を持つ
policy_numbers = np.random.randint(10000000000, 99999999999, num_contracts)
status_codes = np.random.choice(['有効', '無効'], num_contracts, p=[0.8, 0.2])  # 80%は有効な契約


"""
スコアリングデータを生成（基本契約番号に対して一意のスコア）
本来であれば、スコアリングのモデルを構築し、webサイト未登録の契約者にスコアを付与したデータとなる
"""
score_contract_numbers = np.random.choice(np.unique(basic_contract_numbers), num_scores, replace=False)
# スコア分布の調整
# 90-のスコアの発生確率を少なくし、それ以外のスコアの確率を高くする
score_values = np.arange(0, 100)  # スコアの範囲
probabilities = np.ones(100)  # 各スコアの初期確率を1に設定
# 90-97のスコアの確率を減らす
probabilities[90:98] = 0.2  
probabilities /= probabilities.sum()  # 確率の合計を1にするため正規化
# 調整された確率に基づいてスコアを生成
scores = np.random.choice(score_values, num_scores, p=probabilities)
scores = np.random.randint(0, 98, num_scores)


# Webサイト登録者のリストを生成（基本契約番号のみ、重複なし）
registered_contract_numbers = np.random.choice(np.unique(basic_contract_numbers), num_registered, replace=False)


"""
契約データのDataFrameを作成
こちらからwebサイト未登録の契約者を取得する（主人公）
"""
df_contracts = pd.DataFrame({
    '基本契約番号': basic_contract_numbers,
    '証券番号': policy_numbers,
    '契約ステータスコード': status_codes
})

# スコアリングデータのDataFrameを作成
df_scores = pd.DataFrame({
    '基本契約番号': score_contract_numbers,
    '登録スコア': scores
})

"""
Webサイト登録者リストのDataFrameを作成
webサイト未登録の契約者をフィルタリングするために利用
"""
df_registered = pd.DataFrame({
    '基本契約番号': registered_contract_numbers
})

### データのマスク化

In [7]:
df_contracts['基本契約番号'] = df_contracts['基本契約番号'].apply(hash_mask)
df_contracts['証券番号'] = df_contracts['証券番号'].apply(hash_mask)
df_scores['基本契約番号'] = df_scores['基本契約番号'].apply(hash_mask)
df_registered['基本契約番号'] = df_registered['基本契約番号'].apply(hash_mask)

In [8]:
print(df_contracts.shape)
print(df_contracts['証券番号'].nunique())
print(df_contracts['基本契約番号'].nunique())
df_contracts.head()

(15000, 3)
15000
7480


,基本契約番号,証券番号,契約ステータスコード
0,ee57507d74,b893d2a834,無効
1,ee57507d74,17f3e2daef,有効
2,d1c9d4fe9f,b573e81aca,有効
3,d1c9d4fe9f,65498ff111,無効
4,f97b25aba0,82a02f5005,有効


In [9]:
print(df_scores.shape)
print(df_scores['基本契約番号'].nunique())
df_scores.head()

(5000, 2)
5000


,基本契約番号,登録スコア
0,1281b7166e,23
1,4f19431d79,53
2,f8fb31daf3,24
3,15597770aa,24
4,56e68431f1,63


In [10]:
print(df_registered.shape)
print(df_registered['基本契約番号'].nunique())
df_registered.head()

(2500, 1)
2500


,基本契約番号
0,483eeb3510
1,ea62eaef55
2,2c5d027490
3,efe05e50fd
4,5fcfa9da29


In [11]:
df_main = df_contracts.copy()
df_register = df_registered.copy()
df_main.to_csv('../data/df_main.csv', index=False)
df_scores.to_csv('../data/df_scores.csv', index=False)
df_register.to_csv('../data/df_register.csv', index=False)